In [ ]:
# default_exp models

# SecureReqNet Models

## Load SecureReqNet Model
Currently only alpha is available

In [ ]:
#hide
import os

__file__ = (os.getcwd())

In [ ]:
#export
from tensorflow.keras.models import load_model
import os
def load_securereqnet_model(model):
    '''
    Returns a pretrained version of the selected model
    
    Users can select: alpha    
    '''
    path = os.path.join(os.path.dirname(os.path.abspath(__file__)),'pretrained_models',model + ".hdf5")
    # path = 'pretrained_models/' + model + '.hdf5'
    criticality_network_load = load_model(path) #<----- The Model'
    return criticality_network_load

In [ ]:
#export
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dot, Input, Dense, Reshape, LSTM, Conv2D, Flatten, MaxPooling1D, Dropout, MaxPooling2D
from tensorflow.keras.layers import Embedding, Multiply, Subtract
from tensorflow.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Lambda

def create_alpha():
    '''Creates a securereqnet alpha model and compiles it'''
    #BaseLine Architecture <-------
    embeddigs_cols = 100
    max_len_sentences = 618
    input_sh = (618,100,1)
    #Selecting filters? 
    #https://stackoverflow.com/questions/48243360/how-to-determine-the-filter-parameter-in-the-keras-conv2d-function
    #https://stats.stackexchange.com/questions/196646/what-is-the-significance-of-the-number-of-convolution-filters-in-a-convolutional

    N_filters = 128 # <-------- [HyperParameter] Powers of 2 Numer of Features
    K = 2 # <-------- [HyperParameter] Number of Classess

    #baseline_model = Sequential()
    gram_input = Input(shape = input_sh)

    # 1st Convolutional Layer Convolutional Layer (7-gram)
    conv_1_layer = Conv2D(filters=32, input_shape=input_sh, activation='relu', 
                          kernel_size=(7,embeddigs_cols), padding='valid')(gram_input)

    # Max Pooling 
    max_1_pooling = MaxPooling2D(pool_size=((max_len_sentences-7+1),1), strides=None, padding='valid')(conv_1_layer)

    # Fully Connected layer
    fully_connected_1_gram = Flatten()(max_1_pooling)

    fully_connected_1_gram = Reshape((32, 1, 1))(fully_connected_1_gram)

    # 2nd Convolutional Layer (5-gram)
    conv_2_layer = Conv2D(filters=64, kernel_size=(5,1), activation='relu', 
                          padding='valid')(fully_connected_1_gram)

    max_2_pooling = MaxPooling2D(pool_size=((32-5+1),1), strides=None, padding='valid')(conv_2_layer)

    # Fully Connected layer
    fully_connected_2_gram = Flatten()(max_2_pooling)

    fully_connected_2_gram = Reshape((64, 1, 1))(fully_connected_2_gram)

    # 3rd Convolutional Layer (3-gram)
    conv_3_layer =  Conv2D(filters=128, kernel_size=(3,1), activation='relu', 
                          padding='valid')(fully_connected_2_gram)

    # 4th Convolutional Layer (3-gram)
    conv_4_layer = Conv2D(filters=128, kernel_size=(3,1), activation='relu', 
                         padding='valid')(conv_3_layer)

    # 5th Convolutional Layer (3-gram)
    conv_5_layer = Conv2D(filters=64, kernel_size=(3,1), activation='relu', 
                         padding='valid')(conv_4_layer)

    # Max Pooling
    max_5_pooling = MaxPooling2D(pool_size=(58,1), strides=None, padding='valid')(conv_5_layer)

    # Fully Connected layer
    fully_connected = Flatten()(max_5_pooling)

    # 1st Fully Connected Layer
    deep_dense_1_layer = Dense(32, activation='relu')(fully_connected)
    deep_dense_1_layer = Dropout(0.2)(deep_dense_1_layer) # <-------- [HyperParameter]

    # 2nd Fully Connected Layer
    deep_dense_2_layer = Dense(32, activation='relu')(deep_dense_1_layer)
    deep_dense_2_layer = Dropout(0.2)(deep_dense_2_layer) # <-------- [HyperParameter]

    # 3rd Fully Connected Layer
    deep_dense_3_layer = Dense(16, activation='relu')(deep_dense_2_layer)
    deep_dense_3_layer = Dropout(0.2)(deep_dense_3_layer) # <-------- [HyperParameter]

    predictions = Dense(K, activation='softmax')(deep_dense_3_layer)

    #Criticality Model
    criticality_network = Model(inputs=[gram_input],outputs=[predictions]) 
    print(criticality_network.summary())
    
    #Seting up the Model
    criticality_network.compile(optimizer='adam',loss='binary_crossentropy',
                                  metrics=['accuracy'])
    
    return criticality_network


In [ ]:
#export
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from os import path

def fit_model(corpora_train_x,target_train_y,criticality_network,save_model=True,save_history=True,file_path='',model_name=""):
    '''
    Fits a securereqnet model to the data
    
    Returns model, history
    
    @param save_model (bool): Saves the model to the input path
    
    @param save_history (bool): Saves the history to the path
    
    @param file_path (string): Path to where the model and/or history will be saved to
    
    @param model_name (string): Name to save the model as
    '''
    
    filepath = path.join(file_path, model_name + ".hdf5")
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
    mc = ModelCheckpoint(filepath, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    callbacks_list = [es,mc]

    #Model Fitting
    history = criticality_network.fit(
            x = corpora_train_x, 
            y = target_train_y,
            #batch_size=64,
            epochs=2000, #5 <------ Hyperparameter
            validation_split = 0.2,
            callbacks=callbacks_list
    )
    
    #CheckPoints
    #csv_logger = CSVLogger(system+'_training.log')
    # filepath changed from: "alex-adapted-res-003/best_model.hdf5" for testing
    # The folder alex-adapted-res-003 doesn't exist yet in the repository. RC created 08_test in the root folder 
    # manually
    if(save_model):
        criticality_network.save(filepath)
    
    # filepath changed from: 'alex-adapted-res-003/history_training.csv' for testing
    #Saving Training History
    if(save_history):
        df_history = pd.DataFrame.from_dict(history.history)
        df_history.to_csv(path+'/history_training.csv', encoding='utf-8',index=False)
        df_history.head()
    
    return criticality_network, history

In [ ]:
alpha = create_alpha()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 618, 100, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 612, 1, 32)        22432     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 1, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
reshape (Reshape)            (None, 32, 1, 1)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 1, 64)         384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 64)         

In [ ]:
from securereqnet.preprocessing import *
train_x, test_x, train_y,test_y = process_corpora("../data/small_dataset/",True)

Max. Sentence # words: 527
Mix. Sentence # words: 5


In [ ]:
alpha, history= fit_model(train_x, train_y, alpha,save_model=False,save_history=False)

Epoch 1/2000
13/14 [==========================>...] - ETA: 0s - loss: 0.4149 - accuracy: 0.9543
Epoch 00001: val_accuracy improved from -inf to 0.44144, saving model to .hdf5
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: .hdf5/assets
14/14 [==============================] - 1s 97ms/step - loss: 0.3926 - accuracy: 0.9571 - val_loss: 3.0079 - val_accuracy: 0.4414
Epoch 2/2000
13/14 [==========================>...] - ETA: 0s - loss: 0.0432 - accuracy: 0.9808
Epoch 00002: val_accuracy did not improve from 0.44144
14/14 [==============================] - 0s 22ms/step - loss: 0.0406 - accuracy: 0.9819 - val_loss: 8.2951 - val_accuracy: 0.4414
Epoch 3/2000
13/14 [==========================>...] - ETA: 0s - loss: 0.0350 - accuracy: 0.9976
Epoch 00003: val_accuracy did not im

Epoch 27/2000
13/14 [==========================>...] - ETA: 0s - loss: 2.7175e-04 - accuracy: 1.0000
Epoch 00027: val_accuracy did not improve from 0.44144
14/14 [==============================] - 0s 22ms/step - loss: 2.5519e-04 - accuracy: 1.0000 - val_loss: 8.5411 - val_accuracy: 0.4414
Epoch 28/2000
13/14 [==========================>...] - ETA: 0s - loss: 4.7527e-06 - accuracy: 1.0000
Epoch 00028: val_accuracy did not improve from 0.44144
14/14 [==============================] - 0s 23ms/step - loss: 0.0023 - accuracy: 0.9977 - val_loss: 8.5452 - val_accuracy: 0.4414
Epoch 29/2000
13/14 [==========================>...] - ETA: 0s - loss: 8.8821e-06 - accuracy: 1.0000
Epoch 00029: val_accuracy did not improve from 0.44144
14/14 [==============================] - 0s 22ms/step - loss: 8.3415e-06 - accuracy: 1.0000 - val_loss: 8.5534 - val_accuracy: 0.4414
Epoch 30/2000
13/14 [==========================>...] - ETA: 0s - loss: 1.2888e-07 - accuracy: 1.0000
Epoch 00030: val_accuracy did not

13/14 [==========================>...] - ETA: 0s - loss: 2.4786e-06 - accuracy: 1.0000
Epoch 00055: val_accuracy did not improve from 0.44144
14/14 [==============================] - 0s 25ms/step - loss: 2.3294e-06 - accuracy: 1.0000 - val_loss: 8.5667 - val_accuracy: 0.4414
Epoch 56/2000
13/14 [==========================>...] - ETA: 0s - loss: 7.6656e-08 - accuracy: 1.0000
Epoch 00056: val_accuracy did not improve from 0.44144
14/14 [==============================] - 0s 24ms/step - loss: 7.1984e-08 - accuracy: 1.0000 - val_loss: 8.5667 - val_accuracy: 0.4414
Epoch 57/2000
13/14 [==========================>...] - ETA: 0s - loss: 9.3849e-09 - accuracy: 1.0000
Epoch 00057: val_accuracy did not improve from 0.44144
14/14 [==============================] - 0s 25ms/step - loss: 8.8129e-09 - accuracy: 1.0000 - val_loss: 8.5667 - val_accuracy: 0.4414
Epoch 58/2000
12/14 [========================>.....] - ETA: 0s - loss: 1.2489e-04 - accuracy: 1.0000
Epoch 00058: val_accuracy did not improve f

13/14 [==========================>...] - ETA: 0s - loss: 0.0022 - accuracy: 0.9976  
Epoch 00083: val_accuracy did not improve from 0.44144
14/14 [==============================] - 0s 22ms/step - loss: 0.0021 - accuracy: 0.9977 - val_loss: 8.5667 - val_accuracy: 0.4414
Epoch 84/2000
13/14 [==========================>...] - ETA: 0s - loss: 1.7194e-09 - accuracy: 1.0000
Epoch 00084: val_accuracy did not improve from 0.44144
14/14 [==============================] - 0s 21ms/step - loss: 1.6146e-09 - accuracy: 1.0000 - val_loss: 8.5667 - val_accuracy: 0.4414
Epoch 85/2000
13/14 [==========================>...] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 00085: val_accuracy did not improve from 0.44144
14/14 [==============================] - 0s 23ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 8.5667 - val_accuracy: 0.4414
Epoch 86/2000
13/14 [==========================>...] - ETA: 0s - loss: 2.2208e-04 - accuracy: 1.0000
Epoch 00086: val_accuracy did not improve from 0.

## Link to colab
https://colab.research.google.com/drive/1mOXvgvkqCEgrAahyUH9Bw0ZO_nLglNFq